# Data Preparations

This is the notebook used for converting BDD100K dataset into YOLO format.

In [1]:
import os
import json
import shutil
import numpy as np
from tqdm import tqdm
from rich import print
from pathlib import Path

***

## Data folder definitions

In [2]:
ROOT_DIR = Path(os.getcwd())
DATA_DIR = ROOT_DIR / "data"
BASE_DATA_DIR = ROOT_DIR / "base_data"
BASE_DATA_PRFIX = "bdd100k_"

COCO_DATA_DIR = DATA_DIR / "coco_data"
os.makedirs(COCO_DATA_DIR, exist_ok=True)
YOLO_DATA_DIR = DATA_DIR / "yolo_data"
os.makedirs(YOLO_DATA_DIR, exist_ok=True)

***

## Helper functions

In [3]:
def setup_yolo_output_dir(dataset_name: str):
    yolo_output_datadir = YOLO_DATA_DIR / dataset_name
    os.makedirs(yolo_output_datadir, exist_ok=True)
    yolo_label_dir = YOLO_DATA_DIR / dataset_name / "labels"
    os.makedirs(yolo_label_dir, exist_ok=True)
    os.makedirs(yolo_label_dir / "train", exist_ok=True)
    os.makedirs(yolo_label_dir / "val", exist_ok=True)
    yolo_images_dir = YOLO_DATA_DIR / dataset_name / "images"
    os.makedirs(yolo_images_dir, exist_ok=True)
    os.makedirs(yolo_images_dir / "train", exist_ok=True)
    os.makedirs(yolo_images_dir / "val", exist_ok=True)
    print(f"Done setup path for '{dataset_name}'")
    
    return yolo_output_datadir, yolo_label_dir, yolo_images_dir

In [4]:
from json_to_yolo.general_json2yolo import convert_coco_json

def convert_cocojson_to_stdcoco_util(dataset_name: str):
    print(f"Start conversion tasks for dataset: '{dataset_name}'")
    # train set Conversion
    _ = convert_coco_json(
        json_dir=COCO_DATA_DIR / dataset_name / "train",
        save_dir=YOLO_DATA_DIR / dataset_name / "labels" / "train",
        use_segments=False if "seg" not in dataset_name else True,
    )
    
    # trainset Conversion
    _ = convert_coco_json(
        json_dir=COCO_DATA_DIR / dataset_name / "val",
        save_dir=YOLO_DATA_DIR / dataset_name / "labels" / "val",
        use_segments=False if "seg" not in dataset_name else True,
    )

    # All set
    print(f"Finish conversion tasks for dataset: '{dataset_name}'")
    return

In [5]:
import cv2
from pycocotools import mask as cocomask

def rle_to_coco(annotation: dict) -> dict:
    """Transform the rle coco annotation (a single one) into coco style.
    In this case, one mask can contain several polygons, later leading to several `Annotation` objects.
    In case of not having a valid polygon (the mask is a single pixel) it will be an empty list.
    
    Refer to: https://stackoverflow.com/questions/75326066/coco-annotations-convert-rle-to-polygon-segmentation
    
    Parameters
    ----------
    annotation : dict
        rle coco style annotation
    Returns
    -------
    list[dict]
        list of coco style annotations (in dict format)
    """
    tmp_ann = annotation.copy()
    masked_arr = cocomask.decode(annotation["segmentation"])
    contours, _ = cv2.findContours(masked_arr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    segmentations = [
        contour
        for contour in contours if contour.size >= 6
    ]
    seg_coords = [
        seg.astype(float).flatten().tolist()
        for seg in segmentations
    ]
    # Replace the original segmentation object with the new one
    tmp_ann['segmentation'] = seg_coords

    return tmp_ann

***

## Image Datadir Reorganization

In [3]:
# Reformat images
import os

ORIGINAL_IMAGE_DIR = BASE_DATA_DIR / "100k"
NEW_IMAGE_DIR = BASE_DATA_DIR / "100k_images"

if os.path.exists(NEW_IMAGE_DIR / "done_reorganization_flags.json"):
    with open(NEW_IMAGE_DIR / "done_reorganization_flags.json", "r") as f:
        done_reorganization_flags = json.load(f)
else:
    os.makedirs(NEW_IMAGE_DIR / "train", exist_ok=True)
    os.makedirs(NEW_IMAGE_DIR / "val", exist_ok=True)
    os.makedirs(NEW_IMAGE_DIR / "test", exist_ok=True)
    done_reorganization_flags = {
        "train": False,
        "val": False,
        "test": False
    }
    with open(NEW_IMAGE_DIR / "done_reorganization_flags.json", "w+") as f:
        json.dump(done_reorganization_flags, f)

In [4]:
def move_images(image_dir, dest_dir):
    print(f"Moving images from {image_dir} to {dest_dir}")
    image_filenames = [elem for elem in os.listdir(image_dir) if elem.endswith(".jpg")]
    for image_filename in tqdm(image_filenames):
        shutil.move(image_dir / image_filename, dest_dir)
    print("Done.")
    return

In [9]:
# Train
train_image_dir = ORIGINAL_IMAGE_DIR / "train"
print(f"Train image dir: {train_image_dir}")
[elem for elem in os.listdir(train_image_dir) if not elem.endswith(".jpg")]

Train image dir: /Users/ken/Workspaces/PycharmProjects/bdd100k/base_data/100k/train

[]

In [11]:
train_target_image_dir = NEW_IMAGE_DIR / "train"
if not done_reorganization_flags['train']:
    _ = move_images(image_dir=train_image_dir, dest_dir=train_target_image_dir)
    _ = move_images(image_dir=train_image_dir / "trainA", dest_dir=train_target_image_dir)
    _ = move_images(image_dir=train_image_dir / "trainB", dest_dir=train_target_image_dir)
    _ = move_images(image_dir=train_image_dir / "testA", dest_dir=train_target_image_dir)
    _ = move_images(image_dir=train_image_dir / "testB", dest_dir=train_target_image_dir)
    done_reorganization_flags['train'] = True

In [12]:
# Val
val_image_dir = ORIGINAL_IMAGE_DIR / "val"
print(f"Val target image dir: {val_image_dir}")
[elem for elem in os.listdir(val_image_dir) if not elem.endswith(".jpg")]

Val target image dir: /Users/ken/Workspaces/PycharmProjects/bdd100k/base_data/100k/val

[]

In [13]:
val_target_image_dir = NEW_IMAGE_DIR / "val"
if not done_reorganization_flags['val']:
    _ = move_images(image_dir=val_image_dir, dest_dir=val_target_image_dir)
    done_reorganization_flags['val'] = True

In [14]:
# Test
test_image_dir = ORIGINAL_IMAGE_DIR / "test"
print(f"Test target image dir: {test_image_dir}")
[elem for elem in os.listdir(test_image_dir) if not elem.endswith(".jpg")]

Test target image dir: /Users/ken/Workspaces/PycharmProjects/bdd100k/base_data/100k/test

[]

In [15]:
test_target_image_dir = NEW_IMAGE_DIR / "test"
if not done_reorganization_flags['test']:
    _ = move_images(image_dir=test_image_dir, dest_dir=test_target_image_dir)
    _ = move_images(image_dir=test_image_dir / "trainA", dest_dir=test_target_image_dir)
    _ = move_images(image_dir=test_image_dir / "trainB", dest_dir=test_target_image_dir)
    _ = move_images(image_dir=test_image_dir / "testA", dest_dir=test_target_image_dir)
    _ = move_images(image_dir=test_image_dir / "testB", dest_dir=test_target_image_dir)
    done_reorganization_flags['test'] = True

In [16]:
# Image ID Set
train_image_ids = set([elem.split(".")[0] for elem in os.listdir(train_target_image_dir)])
val_image_ids = set([elem.split(".")[0] for elem in os.listdir(val_target_image_dir)])
test_image_ids = set([elem.split(".")[0] for elem in os.listdir(test_target_image_dir)])
full_image_ids = train_image_ids.union(val_image_ids).union(test_image_ids)

In [17]:
len(full_image_ids), len(train_image_ids), len(val_image_ids), len(test_image_ids)

(100000, 70000, 10000, 20000)

In [29]:
# Dump image ids
import json

dump_dict = {
    "train": sorted(list(train_image_ids)),
    "val": sorted(list(val_image_ids)),
    "test": sorted(list(test_image_ids)),
}
with open(BASE_DATA_DIR / "100k_images" / "image_ids_info.json", "w+") as f:
    json.dump(dump_dict, f)

***

## Detection

In [8]:
# Setup yolo data dir
yolo_output_datadir, yolo_label_dir, yolo_images_dir = setup_yolo_output_dir(dataset_name="det_20_labels_trainval")

Done setup path for 'det_20_labels_trainval'

In [19]:
_ = convert_cocojson_to_stdcoco_util(dataset_name="det_20_labels_trainval")

Start conversion tasks for dataset: 'det_20_labels_trainval'

Annotations /Users/ken/Workspaces/PycharmProjects/bdd100k/data/coco_data/det_20_labels_trainval/train/det_train.json: 100%|██████████| 69853/69853 [00:12<00:00, 5429.19it/s]
Annotations /Users/ken/Workspaces/PycharmProjects/bdd100k/data/coco_data/det_20_labels_trainval/val/det_val.json: 100%|██████████| 10000/10000 [00:01<00:00, 5878.76it/s]


Finish conversion tasks for dataset: 'det_20_labels_trainval'

In [16]:
with open(NEW_IMAGE_DIR / "image_ids_info.json", "r") as f:
    image_ids_info = json.load(f)

train_image_ids = set(image_ids_info["train"])
val_image_ids = set(image_ids_info["val"])
test_image_ids = set(image_ids_info["test"])
full_image_ids = train_image_ids.union(val_image_ids).union(test_image_ids)

In [18]:
train_label_ids = set([elem.split(".")[0] for elem in os.listdir(yolo_label_dir / "train")])
val_label_ids = set([elem.split(".")[0] for elem in os.listdir(yolo_label_dir / "val")])
full_label_ids = train_label_ids.union(val_label_ids)

In [17]:
len(full_image_ids), len(train_image_ids), len(val_image_ids), len(test_image_ids)

(100000, 70000, 10000, 20000)

In [19]:
len(full_label_ids), len(train_label_ids), len(val_label_ids)

(79853, 69853, 10000)

In [14]:
print(f"Train label ids not in train image ids: {train_label_ids - train_image_ids}")
print(f"Val label ids not in val image ids: {val_label_ids - val_image_ids}")

Train label ids not in train image ids: set()

Val label ids not in val image ids: set()

In [6]:
dataset_name = "det_20_labels_trainval"
raw_det_train_path = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "det_train.json"
raw_det_val_path = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "det_val.json"

In [7]:
with open(raw_det_train_path, "r") as f:
    raw_det_train = json.load(f)

with open(raw_det_val_path, "r") as f:
    raw_det_val = json.load(f)

In [10]:
print(f"Train set: {len(raw_det_train)}")
print(f"Attributes: {raw_det_train[0].keys()}")

Train set: 69863

Attributes: dict_keys(['name', 'attributes', 'timestamp', 'labels'])

In [13]:
sample_det_image = raw_det_train[0]
print(f"Sample image: {sample_det_image}, label keys: {sample_det_image['labels'][0].keys()}")

Sample image: {'name': '0000f77c-6257be58.jpg', 'attributes': {'weather': 'clear', 'timeofday': 'daytime', 'scene':
'city street'}, 'timestamp': 10000, 'labels': [{'id': '0', 'attributes': {'occluded': False, 'truncated': False, 
'trafficLightColor': 'G'}, 'category': 'traffic light', 'box2d': {'x1': 1125.902264, 'y1': 133.184488, 'x2': 
1156.978645, 'y2': 210.875445}}, {'id': '1', 'attributes': {'occluded': False, 'truncated': False, 
'trafficLightColor': 'G'}, 'category': 'traffic light', 'box2d': {'x1': 1156.978645, 'y1': 136.637417, 'x2': 
1191.50796, 'y2': 210.875443}}, {'id': '2', 'attributes': {'occluded': False, 'truncated': False, 
'trafficLightColor': 'NA'}, 'category': 'traffic sign', 'box2d': {'x1': 1105.66915985699, 'y1': 211.122087, 'x2': 
1170.79037, 'y2': 233.566141}}, {'id': '3', 'attributes': {'occluded': False, 'truncated': True, 
'trafficLightColor': 'NA'}, 'category': 'traffic sign', 'box2d': {'x1': 0.0, 'y1': 0.246631, 'x2': 100.381647, 
'y2': 122.825696}}, {'id': '4', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'NA'}, 
'category': 'car', 'box2d': {'x1': 49.44476737704903, 'y1': 254.530367, 'x2': 357.805838, 'y2': 487.906215}}, 
{'id': '5', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'NA'}, 'category': 'car', 
'box2d': {'x1': 507.82755, 'y1': 221.727518, 'x2': 908.367588, 'y2': 441.0052451528153}}, {'id': '6', 'attributes':
{'occluded': False, 'truncated': True, 'trafficLightColor': 'NA'}, 'category': 'traffic sign', 'box2d': {'x1': 
0.156955, 'y1': 0.809282, 'x2': 102.417429, 'y2': 133.411856}}]}, label keys: dict_keys(['id', 'attributes', 
'category', 'box2d'])

In [23]:
train_labels_arr = np.array([
    list(label['box2d'].values())
    for sample in raw_det_train if "labels" in sample.keys() and sample['labels'] is not None
        for label in sample['labels']
]).reshape(-1,)

In [34]:
train_labels_cat_arr = np.array([
    label['category']
    for sample in raw_det_train if "labels" in sample.keys() and sample['labels'] is not None
        for label in sample['labels']
]).reshape(-1,)

In [26]:
import pandas as pd

pd.Series(train_labels_arr).max(), pd.Series(train_labels_arr).min()

(1280.0, 0.0)

In [35]:
pd.Series(train_labels_cat_arr).unique()

array(['traffic light', 'traffic sign', 'car', 'pedestrian', 'bus',
       'truck', 'rider', 'bicycle', 'motorcycle', 'train',
       'other vehicle', 'other person', 'trailer'], dtype=object)

In [42]:
def process_detection_labels(det_sample: dict, label_dir: str, category_map: dict):
    label_id = det_sample['name'].split(".")[0]
    h, w = 720.0, 1280.0
    
    # Process Labels
    if "labels" not in det_sample.keys() or det_sample['labels'] is None: return
    else: det_labels = det_sample['labels']
    
    label_data = []
    for label in det_labels:
        category = label['category']
        if category in category_map: category_id = category_map[category]
        else: raise ValueError(f"Category '{category}' not found in category map.")

        # Process the box2d
        box2d = label['box2d']
        x1, y1, x2, y2 = box2d['x1'], box2d['y1'], box2d['x2'], box2d['y2']
        x_center, y_center = (x1 + x2) / 2, (y1 + y2) / 2
        width, height = x2 - x1, y2 - y1
        # YOLO BBox is [center x, center y, width, height]
        bbox = np.array([x_center, y_center, width, height], dtype=np.float64)
        bbox[[0, 2]] /= w  # normalize x
        bbox[[1, 3]] /= h  # normalize y
        assert all(bbox >= 0) and all(bbox <= 1), f"Invalid bbox: {bbox}"
        
        label_str_detection = " ".join(
                [str(category_id)] +
                ["{:.4f}".format(elem) for elem in bbox.tolist()]
            ) + "\n"
        label_data += label_str_detection
    # End processing labels
    
    # Save the labels
    det_label_path = label_dir / f"{label_id}.txt"
    with open(det_label_path, "w+") as f:
        f.writelines(label_data)
    # End saving labels
    return

def convert_bbox_detection_dataset(raw_data: dict, dataset_name: str, split: str, category_map: dict):
    # Define paths
    target_label_dir = YOLO_DATA_DIR / dataset_name / "labels" / split
    # Process frames
    for frame in tqdm(raw_data, desc=f"Annotations for '{dataset_name}' {split} set"):
        _ = process_detection_labels(
            frame, 
            label_dir=target_label_dir,
            category_map=category_map
        )
    return len(os.listdir(target_label_dir))

In [43]:
_ = setup_yolo_output_dir(dataset_name=dataset_name)

Done setup path for 'det_20_labels_trainval'

In [44]:
# Load the category map
category_map = {
    "pedestrian": 0,
    "rider": 1,
    "car": 2,
    "truck": 3,
    "bus": 4,
    "train": 5,
    "motorcycle": 6,
    "bicycle": 7,
    "traffic light": 8,
    "traffic sign": 9,
    "other vehicle": 10,
    "other person": 11,
    "trailer": 12
}

num_train_labels = convert_bbox_detection_dataset(
    raw_data=raw_det_train,
    dataset_name="det_20_labels_trainval",
    split="train",
    category_map=category_map
)

Annotations for 'det_20_labels_trainval' train set: 100%|██████████| 69863/69863 [00:19<00:00, 3599.80it/s]


In [45]:
num_val_labels = convert_bbox_detection_dataset(
    raw_data=raw_det_val,
    dataset_name="det_20_labels_trainval",
    split="val",
    category_map=category_map
)

Annotations for 'det_20_labels_trainval' val set: 100%|██████████| 10000/10000 [00:02<00:00, 3727.19it/s]


***

## Our Own BDD100K to COCO

In [6]:
dataset_name = "drivable_labels_trainval"
raw_rle_train_path = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "rles" / "drivable_train.json"
raw_rle_val_path = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "rles" / "drivable_val.json"

In [7]:
with open(raw_rle_train_path, "r") as f:
    drivable_train_rle = json.load(f)
    
with open(raw_rle_val_path, "r") as f:
    drivable_val_rle = json.load(f)

In [8]:
print(f"Attributes: {drivable_train_rle.keys()}")
sample_frame = drivable_train_rle['frames'][0]
print(f"Attributes of 'frame' object: {sample_frame.keys()}, sample frame: {sample_frame}")

Attributes: dict_keys(['frames', 'groups', 'config'])

Attributes of 'frame' object: dict_keys(['name', 'url', 'videoName', 'intrinsics', 'extrinsics', 'attributes', 
'timestamp', 'frameIndex', 'size', 'labels']), sample frame: {'name': '0000f77c-6257be58.jpg', 'url': None, 
'videoName': None, 'intrinsics': None, 'extrinsics': None, 'attributes': None, 'timestamp': None, 'frameIndex': 
None, 'size': None, 'labels': [{'id': '0', 'index': None, 'manualShape': None, 'manualAttributes': None, 'score': 
None, 'attributes': {'areaType': 'direct'}, 'category': 'direct', 'box2d': {'x1': 0.0, 'y1': 367.0, 'x2': 1279.0, 
'y2': 625.0}, 'box3d': None, 'poly2d': None, 'rle': {'counts': 
'U`0f2jc0000000000O1000O10000000O100000O1000O1000000000O01000000000000O10O10000000O10000000O1000O1000000000O0100000
0000000O10O10000000O100000O1000O1000000000000O01000000000O100000O100000O10000000000O0100000000000O1000O100000O10000
000000O0100000000000O100000O1000O1000000000000O10O10000000O1000000000O10O10000000000O100000O1000O1000000000000O10O1
0000000O1000000000000O01000000000O100000000000O010000000000O1000000000O01000000000000O1000000000O01000000000000O100
0000000O01000000000000O10000000000O01000000000O1000000000000O10O10000000O1000000000000O10000000O10O1000000000000O10
000000000O01000000000O1000000000000O1000000000O01000000000000O10000000000O10000000O1000O10000000000O10000000000O100
0000000O10O10000000000O1000000000000O10000000000O1000O10000000O10000000000O10000000000O1000000000000O10O10000000O10
00000000000O10000000000O1000000000000O10000000O10O10000000000O1000000000000O10000000000O1000000000000O10000000000O1
00000000000O010000000000O10000000000O1000000000000O10000000000O1000000000000O10000000000O1000000000000O10000000000O
10000000000O1000000000000O10000000000O1000000000000O10000000000O1000000000000O10000000000O10000000000O1000000000000
O10000000000O1000000000000000000001O00001O00001O00001O00001O0000001O00001O00001O00001O0000001O00001O00001O00001O000
01O0000001O00001O00001O00001O00001O0001O01O00001O00001O0000000000000000000000O1000000000000000000O10000000001O00000
00O10000000000000000O1000000000000000000O100000000000001O000O1000000000000000000O10000000000000000O1000000000001O00
000O1000000000000000000O1000000000001O000O1000000000000000000O100000000000001O000O10000000000000000O100000000000000
0001N1000000000000000000O1000000000001O000O1000000000000000000O10001O0000000000000O100000000000001O0O10000000000000
00000O1000001O00000000000N2O1O1O1O1O1O1O1O2N1N2O1O1O1O1O1O1O1O1N2O1O1O2N1O1O1O1O1N2O1O1O1O1O1O2N1O1N2O1O1O1O1O1O1O1
O1N3N1O1O1O1O1O1O1O100000001O00000000000000000000001O00000000000000000000001O00000000000000000000001O00000000000000
00001O0000000000000000001O00000000000000000010O00000000000000000001O0000000000000000001O0000000000000000001O0000000
000000000001O00000000000000001O000000000000001O00000000000000001O00000001O00000001O00000000000000001O00000000000000
1O00000000000000001O00000000000000001O000000000000001O000000000000001O0000000000001O0000000000000010O00000000000001
O000000000000001O000000000000001O0000000000001O000000000000001O0000000000001O0000000000001O0000000000001O0000000000
001O000001O000001O0000000000001O0000000000001O0000000000001O0000000000001O0000000000001O0000000000001O00000000001O0
000000000001O00000000001O00000000000]D', 'size': [720, 1280]}, 'graph': None}]}

In [9]:
print(drivable_train_rle['groups'])
print(drivable_train_rle['config'])
len(drivable_train_rle['frames']), len(drivable_val_rle['frames'])
len([elem for elem in drivable_train_rle['frames'] if elem['labels']]), len([elem for elem in drivable_val_rle['frames'] if elem['labels']])

None

None

(66921, 9546)

In [10]:
from pycocotools import mask as cocomask

def process_frame_object_segments(frame: dict, label_dir: str, category_map: dict):
    """Process the frame object and save the labels in the label directory."""
    frame_id, _ = frame['name'].split(".")
    
    # Process Labels
    frame_labels = frame['labels']
    
    if not frame_labels:
        # print(f"Frame '{frame_id}' has no labels. Skipping...")
        return
    
    label_data = []
    for label in frame_labels:
        category = label['category']
        if category in category_map:
            category_id = category_map[category]
        else:
            raise RuntimeError(f"Category '{category}' not found in the category map. Skipping...")
        
        # rle obj
        rle_obj = label['rle']
        h, w = rle_obj['size']
        masked_arr = cocomask.decode(rle_obj) # Decode the rle object
        contours, _ = cv2.findContours(masked_arr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        segmentations = [
            contour.astype(np.float32).flatten().tolist()
            for contour in contours
        ]
        scaled_segmentations = [
            (np.array(segmentation).reshape(-1, 2) / np.array([w, h]))
            .reshape(-1)
            .tolist()
            for segmentation in segmentations
        ]
        label_str_segmentations = [
            " ".join(
                [str(category_id)] +
                ["{:.4f}".format(seg) for seg in scaled_seg]
            ) + "\n"
            for scaled_seg in scaled_segmentations
        ]
        label_data += label_str_segmentations
    # End processing loop for labels
    
    # Save the labels
    frame_label_path = label_dir / f"{frame_id}.txt"
    with open(frame_label_path, "w+") as f:
        f.writelines(label_data)
    # End saving labels
    return

In [11]:
def convert_rle_seg_dataset(raw_data: dict, dataset_name: str, split: str, category_map: dict):
    # Define paths
    frames = raw_data['frames']
    target_label_dir = YOLO_DATA_DIR / dataset_name / "labels" / split
    # Process frames
    for frame in tqdm(frames, desc=f"Annotations for '{dataset_name}' {split} set"):
        _ = process_frame_object_segments(
            frame, 
            label_dir=target_label_dir,
            category_map=category_map
        )
    return len(os.listdir(target_label_dir))

In [12]:
_ = setup_yolo_output_dir(dataset_name=dataset_name)

drivable_cat_map = {
    "direct": 0,
    "alternative": 1,
    "background": 2,
    "unknown": 2
}
split = "train"

num_labels = convert_rle_seg_dataset(
    raw_data=drivable_train_rle,
    dataset_name=dataset_name,
    split=split,
    category_map=drivable_cat_map
)

Done setup path for 'drivable_labels_trainval'

Annotations for 'drivable_labels_trainval' train set: 100%|██████████| 70000/70000 [02:25<00:00, 479.49it/s]


In [13]:
split = "val"

num_labels = convert_rle_seg_dataset(
    raw_data=drivable_val_rle,
    dataset_name=dataset_name,
    split=split,
    category_map=drivable_cat_map
)

Annotations for 'drivable_labels_trainval' val set: 100%|██████████| 10000/10000 [00:20<00:00, 491.29it/s]


***

## Lane Labeling

In [6]:
dataset_name = "lane_labels_trainval"
poly_labels_dir = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "polygons"
mask_labels_dir = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "masks"

raw_poly_train_path = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "polygons" / "lane_train.json"
raw_poly_val_path = BASE_DATA_DIR / (BASE_DATA_PRFIX + dataset_name) / "polygons" / "lane_val.json"

In [7]:
with open(raw_poly_train_path, "r") as f:
    lane_poly_train = json.load(f)

with open(raw_poly_val_path, "r") as f:
    lane_poly_val = json.load(f)

In [8]:
print(f"Attributes: {lane_poly_train[0].keys()}")

Attributes: dict_keys(['name', 'labels'])

In [9]:
sample_poly = lane_poly_train[0]
print(f"Attributes of 'frame' object: {sample_poly.keys()}, sample frame: {sample_poly}")

Attributes of 'frame' object: dict_keys(['name', 'labels']), sample frame: {'name': '0000f77c-6257be58.jpg', 
'labels': [{'id': '0', 'attributes': {'laneDirection': 'parallel', 'laneStyle': 'solid', 'laneTypes': 'road curb'},
'category': 'road curb', 'poly2d': [{'vertices': [[503.674413, 373.137193], [357.797732, 374.672737]], 'types': 
'LL', 'closed': False}]}, {'id': '1', 'attributes': {'laneDirection': 'parallel', 'laneStyle': 'solid', 
'laneTypes': 'road curb'}, 'category': 'road curb', 'poly2d': [{'vertices': [[62.973282, 371.601649], [0.0, 
368.53056]], 'types': 'LL', 'closed': False}]}, {'id': '2', 'attributes': {'laneDirection': 'parallel', 
'laneStyle': 'solid', 'laneTypes': 'road curb'}, 'category': 'road curb', 'poly2d': [{'vertices': [[1274.517503, 
376.208281], [905.986941, 371.601649]], 'types': 'LL', 'closed': False}]}]}

In [10]:
sample_label = sample_poly['labels'][0]
print(f"Attributes of 'label' object: {sample_label.keys()}, sample label: {sample_label}")

Attributes of 'label' object: dict_keys(['id', 'attributes', 'category', 'poly2d']), sample label: {'id': '0', 
'attributes': {'laneDirection': 'parallel', 'laneStyle': 'solid', 'laneTypes': 'road curb'}, 'category': 'road 
curb', 'poly2d': [{'vertices': [[503.674413, 373.137193], [357.797732, 374.672737]], 'types': 'LL', 'closed': 
False}]}

In [19]:
def process_poly_object_segments(poly: dict, label_dir: Path, category_map: dict):
    """Process the frame object and save the labels in the label directory."""
    label_id, _ = poly['name'].split(".")
    
    # Process Labels
    if "labels" not in poly.keys(): return
    else: poly_labels = poly['labels']
    
    if not poly_labels:
        # print(f"Frame '{frame_id}' has no labels. Skipping...")
        return
    
    label_data = []
    for label in poly_labels:
        category = label['category']
        if category in category_map:
            category_id = category_map[category]
        else:
            raise RuntimeError(f"Category '{category}' not found in the category map. Skipping...")
        
        # poly obj
        poly_obj = label['poly2d'] # list of vertice objects
        poly_attr = label['attributes']
        h, w = 720, 1280
        
        segmentations = [
            np.array(vertice_obj['vertices']).astype(np.float32) # shape: (2, 2)
            for vertice_obj in poly_obj
        ]
        scaled_segmentations = [
            (segmentation / np.array([w, h]))
            .reshape(-1)
            .tolist()
            for segmentation in segmentations
        ]
        label_str_segmentations = [
            " ".join(
                [str(category_id)] +
                ["{:.4f}".format(seg) for seg in scaled_seg]
            ) + "\n"
            for scaled_seg in scaled_segmentations
        ]
        label_data += label_str_segmentations
    # End processing loop for labels
    
    # Save the labels
    poly_label_path = label_dir / f"{label_id}.txt"
    with open(poly_label_path, "w+") as f:
        f.writelines(label_data)
    # End saving labels
    return

In [20]:
def convert_poly_seg_dataset(raw_data: list, dataset_name: str, split: str, category_map: dict):
    # Define paths
    target_label_dir = YOLO_DATA_DIR / dataset_name / "labels" / split
    # Process frames
    for poly in tqdm(raw_data, desc=f"Annotations for '{dataset_name}' {split} set"):
        _ = process_poly_object_segments(
            poly, 
            label_dir=target_label_dir,
            category_map=category_map
        )
    return len(os.listdir(target_label_dir))

In [21]:
_ = setup_yolo_output_dir(dataset_name=dataset_name)
lane_cat_map = {
    "crosswalk": 0,
    "double other": 1,
    "double white": 2,
    "double yellow": 3,
    "road curb": 4,
    "single other": 5,
    "single white": 6,
    "single yellow": 7,
    "background": 8
}
split = "train"

num_labels = convert_poly_seg_dataset(
    raw_data=lane_poly_train,
    dataset_name=dataset_name,
    split=split,
    category_map=lane_cat_map
)

Done setup path for 'lane_labels_trainval'

Annotations for 'lane_labels_trainval' train set: 100%|██████████| 70000/70000 [00:08<00:00, 8676.64it/s]


In [22]:
split = "val"

num_labels = convert_poly_seg_dataset(
    raw_data=lane_poly_val,
    dataset_name=dataset_name,
    split=split,
    category_map=lane_cat_map
)

Annotations for 'lane_labels_trainval' val set: 100%|██████████| 10000/10000 [00:01<00:00, 9582.79it/s]


In [ ]:
from PIL import Image

def process_poly_mask_segment(poly: dict, label_dir: Path, category_map: dict, mask_dir: Path):
    """Process the frame object and save the labels in the label directory."""
    label_id, _ = poly['name'].split(".")
    
    # Process Labels
    if "labels" not in poly.keys() or poly['labels'] is None: return
    else: poly_labels = poly['labels']
    
    label_data = []
    for label in poly_labels:
        # Category mapping into class index
        category = label['category']
        if category in category_map: category_id = category_map[category]
        else: raise RuntimeError(f"Category '{category}' not found in the category map. Skipping...")
        
        # use mask picture instead of polygon
        poly_mask = Image.open(mask_dir / poly['name']) # mask image name is the same as the original image name
        masked_arr = poly_mask.toarray()
        
        poly_attr = label['attributes']
        h, w = 720, 1280
        
        segmentations = [
            np.array(vertice_obj['vertices']).astype(np.float32) # shape: (2, 2)
            for vertice_obj in poly_obj
        ]
        scaled_segmentations = [
            (segmentation / np.array([w, h]))
            .reshape(-1)
            .tolist()
            for segmentation in segmentations
        ]
        label_str_segmentations = [
            " ".join(
                [str(category_id)] +
                ["{:.4f}".format(seg) for seg in scaled_seg]
            ) + "\n"
            for scaled_seg in scaled_segmentations
        ]
        label_data += label_str_segmentations
    # End processing loop for labels
    
    # Save the labels
    poly_label_path = label_dir / f"{label_id}.txt"
    with open(poly_label_path, "w+") as f:
        f.writelines(label_data)
    # End saving labels
    return

***

## Image Labels

In [6]:
dataset_name = "image_labels"
image_labels_dir = BASE_DATA_DIR / dataset_name
image_train_labels_path = image_labels_dir / "bdd100k_labels_images_train.json"
image_val_labels_path = image_labels_dir / "bdd100k_labels_images_val.json"

In [7]:
with open(image_train_labels_path, "r") as f:
    image_train_labels = json.load(f)
    
with open(image_val_labels_path, "r") as f:
    image_val_labels = json.load(f)

In [8]:
type(image_val_labels)

list

In [10]:
sample_image_lael = image_val_labels[0]
print(f"Attributes of 'frame' object: {sample_image_lael.keys()}, sample image lab el: {sample_image_lael}")

Attributes of 'frame' object: dict_keys(['name', 'attributes', 'timestamp', 'labels']), sample image lab el: 
{'name': 'b1c66a42-6f7d68ca.jpg', 'attributes': {'weather': 'overcast', 'scene': 'city street', 'timeofday': 
'daytime'}, 'timestamp': 10000, 'labels': [{'category': 'traffic sign', 'attributes': {'occluded': False, 
'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 
1000.698742, 'y1': 281.992415, 'x2': 1040.626872, 'y2': 326.91156}, 'id': 0}, {'category': 'traffic sign', 
'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 
'manualAttributes': True, 'box2d': {'x1': 214.613695, 'y1': 172.190058, 'x2': 274.505889, 'y2': 229.586743}, 'id': 
1}, {'category': 'traffic sign', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 
'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 797.314833, 'y1': 313.186265, 'x2': 
829.756437, 'y2': 341.884608}, 'id': 2}, {'category': 'traffic sign', 'attributes': {'occluded': False, 
'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 
652.575363, 'y1': 303.204232, 'x2': 685.016968, 'y2': 315.681772}, 'id': 3}, {'category': 'traffic light', 
'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'green'}, 'manualShape': True, 
'manualAttributes': True, 'box2d': {'x1': 707.476543, 'y1': 311.93851, 'x2': 716.210821, 'y2': 328.159313}, 'id': 
4}, {'category': 'traffic light', 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 
'green'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 626.37253, 'y1': 295.717708, 'x2': 
636.354562, 'y2': 316.929527}, 'id': 5}, {'category': 'traffic light', 'attributes': {'occluded': False, 
'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 
316.929527, 'y1': 289.478938, 'x2': 329.407067, 'y2': 306.947495}, 'id': 6}, {'category': 'traffic sign', 
'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 
'manualAttributes': True, 'box2d': {'x1': 270.762627, 'y1': 288.231184, 'x2': 309.443003, 'y2': 301.956478}, 'id': 
7}, {'category': 'traffic sign', 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'},
'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 220.852466, 'y1': 300.708724, 'x2': 232.082252, 
'y2': 311.938512}, 'id': 8}, {'category': 'car', 'attributes': {'occluded': True, 'truncated': False, 
'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 205.879417, 'y1': 
338.141346, 'x2': 281.992413, 'y2': 388.051507}, 'id': 9}, {'category': 'car', 'attributes': {'occluded': True, 
'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 
47.414654, 'y1': 344.380116, 'x2': 129.76642, 'y2': 400.529049}, 'id': 10}, {'category': 'car', 'attributes': 
{'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True,
'box2d': {'x1': 247.0553, 'y1': 344.380116, 'x2': 346.875624, 'y2': 395.538032}, 'id': 11}, {'category': 'car', 
'attributes': {'occluded': False, 'truncated': True, 'trafficLightColor': 'none'}, 'manualShape': True, 
'manualAttributes': True, 'box2d': {'x1': 0, 'y1': 336.893592, 'x2': 52.40567, 'y2': 403.024556}, 'id': 12}, 
{'category': 'rider', 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': 'none'}, 
'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 650.079856, 'y1': 354.362148, 'x2': 663.805151, 
'y2': 381.812737}, 'id': 13}, {'category': 'motor', 'attributes': {'occluded': True, 'truncated': False, 
'trafficLightColor': 'none'}, 'manualShape': True, 'manualAttributes': True, 'box2d': {'x1': 648.832101, 'y1': 
366.83969, 'x2': 663.805151, 'y2'

In [ ]:
{
    'name': 'b1c66a42-6f7d68ca.jpg', 
    'attributes': {'weather': 'overcast', 'scene': 'city street', 'timeofday': 'daytime'}, 
    'timestamp': 10000,
}